In [1]:
# import 
import pandas as pd;
import numpy as np;

In [5]:
# Datasets
X1 = pd.read_csv('../datasets/complete-data/Caltech/GISTFeature.csv',header=None);
X2 = pd.read_csv('../datasets/complete-data/Caltech/LBPFeature.csv',header=None);
print(X1.shape) # (1474, 512)
print(X2.shape) # (1474,928)
X1.head()

(1474, 512)
(1474, 928)


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.149860,0.049631,0.049833,0.030009,0.166100,0.045195,0.052351,0.032841,0.072760,0.067054,...,0.045340,0.038174,0.068873,0.088719,0.060006,0.046778,0.024452,0.050038,0.096785,0.070728
1,0.168870,0.082086,0.167050,0.109790,0.113190,0.045916,0.050687,0.031878,0.207420,0.102760,...,0.118360,0.075107,0.081991,0.046442,0.163500,0.110060,0.112140,0.085244,0.079044,0.051326
2,0.031449,0.038602,0.040515,0.064938,0.024486,0.064542,0.038623,0.055522,0.035708,0.034724,...,0.040049,0.055716,0.054746,0.068624,0.043013,0.089594,0.009888,0.060689,0.055650,0.049560
3,0.062685,0.069544,0.061907,0.038404,0.052422,0.041499,0.053826,0.048148,0.034058,0.040677,...,0.085608,0.071643,0.119500,0.058825,0.082927,0.062791,0.048369,0.044633,0.069152,0.102740
4,0.062920,0.034458,0.015966,0.022567,0.127380,0.098888,0.062020,0.031535,0.049430,0.058208,...,0.029204,0.038763,0.047977,0.071451,0.072366,0.055249,0.028656,0.041247,0.110050,0.054509


In [54]:
# Normalize
from sklearn.preprocessing import Normalizer
transformer = Normalizer();
Xn1 = transformer.transform(X1).T
Xn2 = transformer.transform(X2).T

In [55]:
np.random.seed(123)
import numpy.matlib
Ht = np.matlib.rand(V[0].shape)

In [56]:
V = [Xn1,Xn2];

In [68]:
V[0].shape  # 10x 1474
# H => 10 x 1474
# w => 1474 x 10


(512, 1474)

In [ ]:
# 512 x 1474 => (512 x 10) x (10 x 1474)
#      X     =>   W      x   H

In [182]:
# random H1,H2, W1,W2 # k = 5
H=[]
W=[]
for i in range(len(V)):
    sh = V[i].shape
    Wt = np.matlib.rand(sh[0],10)
    Ht = np.matlib.rand(sh[1],10).T
    W.append(Wt);
    H.append(Ht);
    

In [170]:
print(W[0].shape)
print(H[0].shape)

(512, 10)
(10, 1474)


In [7]:
# update H
# 1474 x something

# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)
# H = list [10 x 1470][10 x 1470]
# view =  0

# up => (10 x 512 )(512 x 1474) => (10 x 1474)
# mid => (10  x 14704 )
# other => (10 x 512)(512x 10)(10 x 1470) => (10 x 1474)
# mid + other > (10 x 1470)
# final > (10 x 1470)


def h_coefficient_matrix(W,X,HF,alpha,beta,H,view_no):
    # Upper calculation
    up= 2*(W.T)*X # 512 x 1474 * 1474 x 512 => 512 x 512
    # for middle loop
    mid_H=0;
    for i in range(len(H)): # [H0, H1]
        if(i == view_no):
            continue;
        mid_H = mid_H + H[i];
    mid_H = alpha*mid_H;
    down = (2*W.T*(W)*HF)+2*beta*HF+mid_H; # H => previous H
    final = up/down;
    return final;

In [8]:
# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)
# H = list [10 x 1470][10 x 1470]
# view =  0

# up => 512x1474 ) (1474 x 10 ) => 512 x 10
# dow => 512 x 10 )( 10x1474 ) (1474 x 10) => 512 x 10

def w_cofficient_matrix(X,H,W):
    up = X * (H.T);
    down = W * H * (H.T)
    final = up/down
    return final;
    
    

In [10]:
from sklearn.metrics.pairwise import euclidean_distances
x = np.array([[1,2],[3,4.5]])
y = np.array([[1,2.1],[3,4]])
d = euclidean_distances(x,y)
di = [];
for i in range(2):
    di.append(d[i][i])
di
max(di)

0.5

In [124]:
a = V[0].shape
a[0]

512

In [164]:
np.multiply(H[0],H[1]).shape

(10, 1474)

In [5]:
# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)

def findError(X,W,H):
    loop = X.shape
    loop = loop[1];
    sumz = 0;
    L =W*H ; # 512 x 10 10x 1474 => 512 x 1474
    L = L.T;
    X = X.T;
    error = euclidean_distances(X,L);
    DI= [];
    for i in range(loop):
        DI.append(error[i][i])
        
    return max(DI),np.average(DI);

In [184]:
alpha = 0.0001;
beta = 0.0001;
iteration = 0;

for index,X in enumerate(V): # X1, X2
    max_E , avg_E = findError(X,W[index],H[index]);
    while( max_E >= 0.566): # 0.5 
        # updata H 
        H[index] =  np.multiply(H[index],h_coefficient_matrix(W[index],X,H[index],alpha,beta,H,index));
        # update W
        W[index] =  np.multiply(W[index],w_cofficient_matrix(X,H[index],W[index]));
        # increase number of iteration
        iteration = iteration + 1;
        # display info
        max_E , avg_E = findError(X,W[index],H[index]);
        print(f"Error:: of Iteration {iteration} => ","Maximum Error: ",max_E," &&& Average Error:",avg_E);
        # End of View or End of While Loop;
    # inside For Loop for View
    print('End of View ',index)
    

End of View  0
Error:: of Iteration 1 =>  Maximum Error:  0.751517807097332  &&& Average Error: 0.35438022162617594
Error:: of Iteration 2 =>  Maximum Error:  0.6979994206838757  &&& Average Error: 0.33770054822138745
Error:: of Iteration 3 =>  Maximum Error:  0.6939661073771007  &&& Average Error: 0.3355179157595168
Error:: of Iteration 4 =>  Maximum Error:  0.6899264035419238  &&& Average Error: 0.3333945430288152
Error:: of Iteration 5 =>  Maximum Error:  0.6849025416893237  &&& Average Error: 0.3310956330389493
Error:: of Iteration 6 =>  Maximum Error:  0.6784549063826059  &&& Average Error: 0.3284326542644478
Error:: of Iteration 7 =>  Maximum Error:  0.6701242122794869  &&& Average Error: 0.3251911503218026
Error:: of Iteration 8 =>  Maximum Error:  0.6594635979756991  &&& Average Error: 0.32113802383597395
Error:: of Iteration 9 =>  Maximum Error:  0.6461736408030081  &&& Average Error: 0.3160565964373361
Error:: of Iteration 10 =>  Maximum Error:  0.6303215231479009  &&& Averag

In [213]:
# find avearage of H => H*

# initialize all to 0 with H dimenstion
sum_H = np.zeros(H[0].shape);

for h in H:
    sum_H = sum_H + h;

# Average of all H ;
H_star = sum_H/len(H);


matrix([[0.00865075, 0.00817312, 0.00267336, ..., 0.00063916, 0.00151358,
         0.00358691],
        [0.01105359, 0.00723174, 0.01081549, ..., 0.00418431, 0.0073252 ,
         0.00718541],
        [0.00688953, 0.01228485, 0.00795208, ..., 0.00669529, 0.0059402 ,
         0.0021677 ]])

## Overall Coding
1. Load Data
2. Normalize
3. Randomize H and W
4. Find H and W
5. Averaging H

> input => multiview dataset V, alpha , beta;

> output => H*

In [1]:
# 0. Import All requirements
import pandas as pd;
import numpy as np;
import numpy.matlib;
from sklearn.preprocessing import Normalizer

In [2]:
# 1 & 2. Load Data and Normalize
#
# LOAD
# 
X1 = pd.read_csv('../datasets/complete-data/Caltech/GISTFeature.csv',header=None);
X2 = pd.read_csv('../datasets/complete-data/Caltech/LBPFeature.csv',header=None);
# NORMALIZE
#
transformer = Normalizer();
Xn1 = transformer.transform(X1).T
Xn2 = transformer.transform(X2).T
# COMBINE
#
V = [Xn1,Xn2];

In [14]:
# 3. Random value of H and W for all View
num_of_dimension = 10;  # Change this to test with other dimension :
H=[]
W=[]

for i in range(len(V)):
    sh = V[i].shape
    Wt = np.matlib.rand(sh[0],num_of_dimension)
    Ht = np.matlib.rand(sh[1],num_of_dimension).T
    W.append(Wt);
    H.append(Ht);
# End of H and W Randomization

In [15]:
# 4. Update H and W for all View
alpha = 0.0001; # parameter
beta = 0.0001;  # parameter 
iteration = 0;  # tracking of iteration

for index,X in enumerate(V): # X1, X2
    max_E , avg_E = findError(X,W[index],H[index]);
    while( max_E >= 0.566): # Effect by random value of H , W ;
        # updata H 
        H[index] =  np.multiply(H[index],h_coefficient_matrix(W[index],X,H[index],alpha,beta,H,index));
        # update W
        W[index] =  np.multiply(W[index],w_cofficient_matrix(X,H[index],W[index]));
        # increase number of iteration
        iteration = iteration + 1;
        # display info
        max_E , avg_E = findError(X,W[index],H[index]);
        print(f"Error:: of Iteration {iteration} => ","Maximum Error: ",max_E," &&& Average Error:",avg_E);
        # End of View or End of While Loop;
    # inside For Loop for View
    print('End of View ',index)
    iteration = 0;

Error:: of Iteration 1 =>  Maximum Error:  0.8628712518999498  &&& Average Error: 0.4519839235401925
Error:: of Iteration 2 =>  Maximum Error:  0.8579897814003695  &&& Average Error: 0.44705776236865896
Error:: of Iteration 3 =>  Maximum Error:  0.8575514601207841  &&& Average Error: 0.4459614669187957
Error:: of Iteration 4 =>  Maximum Error:  0.8571087544958778  &&& Average Error: 0.4449895895677228
Error:: of Iteration 5 =>  Maximum Error:  0.8566875102068761  &&& Average Error: 0.44407102685281324
Error:: of Iteration 6 =>  Maximum Error:  0.8562676589352627  &&& Average Error: 0.4431621201607246
Error:: of Iteration 7 =>  Maximum Error:  0.855827309536202  &&& Average Error: 0.4422164361089687
Error:: of Iteration 8 =>  Maximum Error:  0.8553430193362042  &&& Average Error: 0.4411780162683457
Error:: of Iteration 9 =>  Maximum Error:  0.854788141569047  &&& Average Error: 0.43997459456935467
Error:: of Iteration 10 =>  Maximum Error:  0.8541319582026379  &&& Average Error: 0.43850

Error:: of Iteration 83 =>  Maximum Error:  0.6600906623707221  &&& Average Error: 0.30179124567135146
Error:: of Iteration 84 =>  Maximum Error:  0.6584464667743408  &&& Average Error: 0.301538537517594
Error:: of Iteration 85 =>  Maximum Error:  0.6568362575989172  &&& Average Error: 0.30129582014590495
Error:: of Iteration 86 =>  Maximum Error:  0.6552595543554656  &&& Average Error: 0.3010625735183623
Error:: of Iteration 87 =>  Maximum Error:  0.6537158162117652  &&& Average Error: 0.30083830276588674
Error:: of Iteration 88 =>  Maximum Error:  0.6522044316190192  &&& Average Error: 0.30062253772109715
Error:: of Iteration 89 =>  Maximum Error:  0.6507247154071486  &&& Average Error: 0.3004148322263253
Error:: of Iteration 90 =>  Maximum Error:  0.6492759123304653  &&& Average Error: 0.3002147633316936
Error:: of Iteration 91 =>  Maximum Error:  0.6478572056543639  &&& Average Error: 0.3000219304935407
Error:: of Iteration 92 =>  Maximum Error:  0.6464677292277287  &&& Average Err

Error:: of Iteration 164 =>  Maximum Error:  0.5879212906489474  &&& Average Error: 0.2942057675572198
Error:: of Iteration 165 =>  Maximum Error:  0.5874428495616015  &&& Average Error: 0.2941733937737498
Error:: of Iteration 166 =>  Maximum Error:  0.5869704641399682  &&& Average Error: 0.29414152757070366
Error:: of Iteration 167 =>  Maximum Error:  0.5865040744248283  &&& Average Error: 0.29411015759027603
Error:: of Iteration 168 =>  Maximum Error:  0.5860436218703441  &&& Average Error: 0.29407927314387006
Error:: of Iteration 169 =>  Maximum Error:  0.5855890494661006  &&& Average Error: 0.2940488642076151
Error:: of Iteration 170 =>  Maximum Error:  0.585140301766833  &&& Average Error: 0.2940189213917311
Error:: of Iteration 171 =>  Maximum Error:  0.5846973248103458  &&& Average Error: 0.29398943588491766
Error:: of Iteration 172 =>  Maximum Error:  0.5842600659156646  &&& Average Error: 0.29396039937748647
Error:: of Iteration 173 =>  Maximum Error:  0.5838284733663351  &&& 

Error:: of Iteration 11 =>  Maximum Error:  0.6658905366841805  &&& Average Error: 0.29780567162805954
Error:: of Iteration 12 =>  Maximum Error:  0.6495679191636453  &&& Average Error: 0.28887057103353064
Error:: of Iteration 13 =>  Maximum Error:  0.6320269990622737  &&& Average Error: 0.27987668278396094
Error:: of Iteration 14 =>  Maximum Error:  0.614770216717475  &&& Average Error: 0.2715408472143692
Error:: of Iteration 15 =>  Maximum Error:  0.599049875400672  &&& Average Error: 0.2643805489560277
Error:: of Iteration 16 =>  Maximum Error:  0.5899020313921732  &&& Average Error: 0.25859041502670965
Error:: of Iteration 17 =>  Maximum Error:  0.5850686237592688  &&& Average Error: 0.2540840985857293
Error:: of Iteration 18 =>  Maximum Error:  0.5809694640230199  &&& Average Error: 0.2506234226620843
Error:: of Iteration 19 =>  Maximum Error:  0.5774310954306923  &&& Average Error: 0.24793751081946483
Error:: of Iteration 20 =>  Maximum Error:  0.5742953135719887  &&& Average Err

In [16]:
# Averaging
sum_H = np.zeros(H[0].shape);
for h in H:
    sum_H = sum_H + h;
H_star = sum_H/len(H);  # <=======================   output -  H*


In [17]:
H_star.shape # (reduced dimension x sample number )

(10, 1474)

In [18]:
Label = pd.read_csv("../datasets/complete-data/Caltech/label.csv",header=None);
from sklearn.cluster import KMeans
Label = Label.to_numpy()
Label.shape

(1474, 1)

In [19]:
km = KMeans(n_clusters=7,random_state=0).fit(H_star.T);

In [20]:
y_pred = km.predict(H_star.T)

In [21]:
from sklearn.metrics import accuracy_score;

In [22]:
accuracy_score(Label,y_pred)

0.1919945725915875